# TensorFlow实现简单CNN网络
使用TensorFlow实现一个简单的CNN网络，用于识别MNIST数据

In [125]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
# 读入mnist数据,以独热向量读入
mnist = input_data.read_data_sets("./data/MNIST_data/", one_hot=True)
img = mnist.train.images[500]
print(img.shape)
x = tf.placeholder(tf.float32,[None, 784])
y_ = tf.placeholder(tf.float32,[None, 10])


Extracting ./data/MNIST_data/train-images-idx3-ubyte.gz
Extracting ./data/MNIST_data/train-labels-idx1-ubyte.gz
Extracting ./data/MNIST_data/t10k-images-idx3-ubyte.gz
Extracting ./data/MNIST_data/t10k-labels-idx1-ubyte.gz
(784,)


In [126]:
#784-->28*28
x_image = tf.reshape(x,[-1,28,28,1])


In [127]:
def weight_variable(shape):
    '''
    初始化权重w
    '''
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    '''
    初始化偏置b
    '''
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)
    
def conv2d(x, W):
    '''
    实现卷积层
    '''
    return tf.nn.conv2d(x, W, strides=[1,1,1,1], padding='SAME')

def max_pool_2x2(x):
    '''
    实现池化层
    '''
    #padding='SAME'表示通过填充0，使得输入和输出的形状一致
    return tf.nn.max_pool(x, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')

In [128]:
# 第一层 -> 卷积层
W_conv1 = weight_variable([5,5,1,32])
b_conv1 = bias_variable([32])
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)#池化

# 第二层 -> 卷积层
W_conv2 = weight_variable([5,5,32,64])
b_conv2 = bias_variable([64])
h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)#池化

In [129]:
# 全连接层
W_fc1 = weight_variable([7*7*64, 1024])
b_fc1 = bias_variable([1024])
h_pool2_flat = tf.reshape(h_pool2, [-1,7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1)+b_fc1)
keep_prob = tf.placeholder(tf.float32)
h_fcl_drop = tf.nn.dropout(h_fc1, keep_prob)

In [130]:
#输出层
W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])
y_conv = tf.matmul(h_fcl_drop, W_fc2)+b_fc2

In [131]:
#使用了v2版本的cross_entropy https://blog.csdn.net/tsyccnh/article/details/81069308
cross_entropy = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y_conv))
#也可以将0.0001写成le-4
train_step = tf.train.AdamOptimizer(0.0001).minimize(cross_entropy)

In [135]:
correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_,1))
ayutest = tf.cast(correct_prediction,tf.float32)
accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))

In [138]:
#创建Session，对变量进行初始化
with tf.Session() as sess:

    sess.run(tf.initialize_all_variables())

    for i in range(20000):
        batch = mnist.train.next_batch(50)

        if i % 100 == 0:
            train_accuacy = accuracy.eval(feed_dict={x: batch[0], y_: batch[1], keep_prob: 1.0})
            print("step %d, training accuracy %g"%(i, train_accuacy))
            print(sess.run(y_conv,feed_dict = {x: batch[0], y_: batch[1], keep_prob: 0.5}))
            print(sess.run(correct_prediction,feed_dict={x: batch[0], y_: batch[1], keep_prob: 1.0}))
            print(sess.run(ayutest,feed_dict={x: batch[0], y_: batch[1], keep_prob: 1.0}))
            
        train_step.run(feed_dict = {x: batch[0], y_: batch[1], keep_prob: 0.5})

step 0, training accuracy 0.06
[[ 10.2680855    3.5203443  -13.864965    -2.531102    -6.0521965
   -2.011364    -0.5072402   -4.9584026    0.8382305    7.77655   ]
 [ 10.839143     0.05961571 -16.529726     1.7026372   -3.074683
    0.64425623 -10.731301     2.5965278  -10.411194     6.3535895 ]
 [ 10.303163    -1.4619964  -16.515627    -7.1948414  -13.822494
    0.258885    -5.205549    -5.546969    -3.568048     2.5307722 ]
 [  6.760332    -3.3292408  -13.830477    -6.182302    -9.0087595
    4.349248     3.6291277    5.420248    -7.1270585    4.7238507 ]
 [  8.920841     5.1553955   -3.7497988    1.4137195   -6.1872864
    3.5115764    2.207295    -2.7879457  -10.956324     1.2517246 ]
 [ -4.606278    -2.080823    -3.2957108    1.6339535   -9.960443
   -9.388582    -6.4011436   -0.7767118   -4.5742927    0.3943263 ]
 [ 10.240614    -0.15951124 -13.551132     2.043434    -3.9504485
   -4.6045156    7.7626023   -4.7500305   -9.011856     2.4802666 ]
 [  7.539684    -5.399715   -10.19

KeyboardInterrupt: 

In [71]:
#运行完后，释放对话内存
sess.close()

In [57]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

def weight_varible(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')



mnist = input_data.read_data_sets("./data/MNIST_data/", one_hot=True)
print("Download Done!")

sess = tf.InteractiveSession()

# paras
W_conv1 = weight_varible([5, 5, 1, 32])
b_conv1 = bias_variable([32])

# conv layer-1
x = tf.placeholder(tf.float32, [None, 784])
x_image = tf.reshape(x, [-1, 28, 28, 1])

h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

# conv layer-2
W_conv2 = weight_varible([5, 5, 32, 64])
b_conv2 = bias_variable([64])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

# full connection
W_fc1 = weight_varible([7 * 7 * 64, 1024])
b_fc1 = bias_variable([1024])

h_pool2_flat = tf.reshape(h_pool2, [-1, 7 * 7 * 64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

# dropout
keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

# output layer: softmax
W_fc2 = weight_varible([1024, 10])
b_fc2 = bias_variable([10])

y_conv = tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)
y_ = tf.placeholder(tf.float32, [None, 10])

# model training
# cross_entropy = -tf.reduce_sum(y_ * tf.log(y_conv))
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_,logits=y_conv))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

correct_prediction = tf.equal(tf.arg_max(y_conv, 1), tf.arg_max(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

sess.run(tf.initialize_all_variables())

for i in range(20000):
    batch = mnist.train.next_batch(50)

    if i % 100 == 0:
        train_accuacy = accuracy.eval(feed_dict={x: batch[0], y_: batch[1], keep_prob: 1.0})
        print("step %d, training accuracy %g"%(i, train_accuacy))
    train_step.run(feed_dict = {x: batch[0], y_: batch[1], keep_prob: 0.5})

# accuacy on test
print("test accuracy %g"%(accuracy.eval(feed_dict={x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0})))

Extracting ./data/MNIST_data/train-images-idx3-ubyte.gz
Extracting ./data/MNIST_data/train-labels-idx1-ubyte.gz
Extracting ./data/MNIST_data/t10k-images-idx3-ubyte.gz
Extracting ./data/MNIST_data/t10k-labels-idx1-ubyte.gz
Download Done!


/Users/ayuliao/.pyenv/versions/anaconda3-4.4.0/envs/tensorflow_py36/lib/python3.6/site-packages/tensorflow/python/client/session.py:1714: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


step 0, training accuracy 0.06
step 100, training accuracy 0.58
step 200, training accuracy 0.6


KeyboardInterrupt: 

In [44]:
import tensorflow as tf

In [45]:
dropout = tf.placeholder(tf.float32)

In [46]:
x = tf.Variable(tf.ones([10,10]))
y = tf.nn.dropout(x, dropout)

In [47]:
init = tf.initialize_all_variables()
sess = tf.Session()
sess.run(init)

In [48]:
#0.4为存活概率
print(sess.run(y, feed_dict={dropout:0.4}))

[[2.5 0.  0.  0.  0.  0.  0.  0.  0.  0. ]
 [0.  2.5 2.5 0.  2.5 2.5 0.  2.5 2.5 2.5]
 [2.5 2.5 0.  2.5 0.  0.  2.5 0.  0.  0. ]
 [2.5 0.  0.  0.  2.5 0.  0.  0.  2.5 0. ]
 [2.5 0.  0.  0.  0.  2.5 2.5 2.5 0.  0. ]
 [0.  2.5 0.  0.  0.  0.  0.  2.5 0.  2.5]
 [0.  0.  0.  2.5 2.5 2.5 0.  2.5 0.  0. ]
 [0.  2.5 0.  2.5 2.5 2.5 0.  2.5 0.  0. ]
 [2.5 2.5 0.  2.5 0.  0.  2.5 2.5 0.  2.5]
 [2.5 0.  2.5 2.5 0.  0.  2.5 0.  0.  0. ]]


In [50]:
z = tf.layers.dropout(x, rate=dropout, training=True)
init = tf.initialize_all_variables()
sess.run(init)
print(sess.run(z, feed_dict={dropout:0.6}))

[[0.        2.5000002 0.        2.5000002 2.5000002 2.5000002 0.
  0.        0.        2.5000002]
 [2.5000002 0.        0.        0.        0.        0.        0.
  0.        0.        0.       ]
 [2.5000002 2.5000002 0.        0.        0.        0.        0.
  2.5000002 0.        0.       ]
 [0.        2.5000002 2.5000002 2.5000002 0.        0.        0.
  0.        2.5000002 2.5000002]
 [2.5000002 0.        2.5000002 0.        0.        0.        2.5000002
  2.5000002 0.        0.       ]
 [0.        0.        2.5000002 0.        0.        0.        0.
  0.        0.        2.5000002]
 [2.5000002 2.5000002 0.        0.        2.5000002 2.5000002 0.
  2.5000002 0.        0.       ]
 [0.        0.        0.        2.5000002 0.        0.        2.5000002
  0.        2.5000002 2.5000002]
 [2.5000002 2.5000002 0.        0.        2.5000002 0.        0.
  0.        2.5000002 2.5000002]
 [2.5000002 0.        0.        0.        2.5000002 2.5000002 2.5000002
  2.5000002 0.        0.       ]

In [35]:
x = tf.Variable(tf.ones([10,10]))
print(sess.run(x))


FailedPreconditionError: Attempting to use uninitialized value Variable_13
	 [[Node: _retval_Variable_13_0_0 = _Retval[T=DT_FLOAT, index=0, _device="/job:localhost/replica:0/task:0/device:CPU:0"](Variable_13)]]

In [115]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(20000):
        batch = mnist.train.next_batch(50)
        # 每100轮计算一次准确率
        if i%100 == 0:
            #训练时的精度
            train_accuracy = sess.run(accuracy, feed_dict={x:batch[0],y_:batch[1], keep_prob:1.0})
            print('step %d, training accuracy %g'%(i, train_accuracy))
            print('correct_prediction %s'%sess.run(correct_prediction,{x:batch[0],y_:batch[1], keep_prob:1.0}))
        sess.run(train_step,feed_dict={x:batch[0], y_:batch[1], keep_prob:0.5})
test_accuracy = sess.run(accuracy, feed_dict={x:mnist.test.images, y_:mnist.test.labels, keep_prob:1.0})
print('测试集准确率: %g'%test_accuracy)
            

step 0, training accuracy 0.08
correct_prediction [False False False False False False False False False False False False
 False  True False False False False False False False False False False
 False False False False False False False False False False  True False
  True False False False False False False False  True False False False
 False False]
step 100, training accuracy 0.84
correct_prediction [ True  True  True  True  True  True  True False  True False  True  True
  True  True  True  True  True  True  True False  True  True  True  True
  True  True  True False  True  True False  True False  True  True  True
  True  True  True  True  True  True  True  True  True  True False  True
 False  True]


KeyboardInterrupt: 

In [62]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(20000):
      batch = mnist.train.next_batch(50)
      if i%100 == 0:
        train_accuracy = accuracy.eval(feed_dict={
            x:batch[0], y_: batch[1], keep_prob: 1.0})
        print("step %d, training accuracy %g"%(i, train_accuracy))
      train_step.run(feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})

ValueError: Cannot feed value of shape (50,) for Tensor 'Placeholder_8:0', which has shape '(?, 10)'

In [63]:
#创建Session，对变量进行初始化
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())

#训练20000步
for i in range(20000):
    batch = mnist.train.next_batch(50)
    if i % 100 == 0:
        train_accuracy = accuracy.eval(feed_dict={
            x:batch[0], y_:batch[1], keep_prob:1.0
        })
        print('步数:[%d] 准确率:[%g]'%(i, train_accuracy))
    train_step.run(feed_dict={x:batch[0], y_:batch[1],keep_prob:0.5})

ValueError: Cannot feed value of shape (50,) for Tensor 'Placeholder_8:0', which has shape '(?, 10)'

In [ ]:
t_y = 